In [1]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import json
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
vocab_size = 10000
embedding_dim = 16
max_length = 30
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 136054
file_path = os.path.join(os.path.dirname(os.getcwd()),"data\\cleaned_data_en_4file.json")
model_save_path = os.path.join(os.path.dirname(os.getcwd()),"models\\twitter_LSTM_model")

In [4]:
with open(file_path, 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['text'])
    labels.append(item['viral'])

In [5]:
print("sentences: ", len(sentences))
print("labels", labels[1:10])

sentences:  170068
labels [0, 0, 1, 0, 0, 0, 0, 0, 0]


In [6]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [7]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [8]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.GlobalAveragePooling1D(),
    # tf.keras.layers.Dense(24, activation='relu'),
    # tf.keras.layers.Dense(1, activation='sigmoid')
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
sgdm_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='binary_crossentropy',optimizer=sgdm_optimizer,metrics=['accuracy'])

NameError: name 'tf' is not defined

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 16)            160000    
                                                                 
 bidirectional (Bidirectiona  (None, 30, 128)          41472     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 246,913
Trainable params: 246,913
Non-tr

In [16]:
num_epochs = 20
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/20
938/938 - 52s - loss: 0.0891 - accuracy: 0.9698 - val_loss: 0.0382 - val_accuracy: 0.9917 - 52s/epoch - 56ms/step
Epoch 2/20
938/938 - 57s - loss: 0.0480 - accuracy: 0.9832 - val_loss: 0.0373 - val_accuracy: 0.9918 - 57s/epoch - 61ms/step
Epoch 3/20
938/938 - 60s - loss: 0.0450 - accuracy: 0.9833 - val_loss: 0.0355 - val_accuracy: 0.9920 - 60s/epoch - 64ms/step
Epoch 4/20
938/938 - 60s - loss: 0.0422 - accuracy: 0.9839 - val_loss: 0.0349 - val_accuracy: 0.9915 - 60s/epoch - 64ms/step
Epoch 5/20
938/938 - 57s - loss: 0.0418 - accuracy: 0.9839 - val_loss: 0.0357 - val_accuracy: 0.9921 - 57s/epoch - 61ms/step
Epoch 6/20
938/938 - 62s - loss: 0.0408 - accuracy: 0.9843 - val_loss: 0.0380 - val_accuracy: 0.9910 - 62s/epoch - 66ms/step
Epoch 7/20
938/938 - 37s - loss: 0.0448 - accuracy: 0.9839 - val_loss: 0.0365 - val_accuracy: 0.9922 - 37s/epoch - 40ms/step
Epoch 8/20
938/938 - 33s - loss: 0.0411 - accuracy: 0.9841 - val_loss: 0.0375 - val_accuracy: 0.9918 - 33s/epoch - 36ms/step


In [17]:
model.save("twitter_LSTM_model")

INFO:tensorflow:Assets written to: twitter_LSTM_model\assets


INFO:tensorflow:Assets written to: twitter_LSTM_model\assets


In [20]:
sentence = ["If the Cleveland Cavaliers win the 2018 NBA finals I’ll buy everyone who retweet’s this a jersey..."]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[3.025972e-07]]
